In [1]:
from lxml import etree
from PIL import Image
import cv2,os
from shutil import copyfile

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
from numpy import nan
import pandas as pd
import imghdr            #library for fetching image file type

In [2]:
imgset=[]
for (root, dirs, files) in os.walk('dataset/',topdown=True):
    for f in dirs:
        if 'img' in f:
            imgset.append(f)
imgset.sort()
print(imgset)
print('\nTotal {} folders'.format(len(imgset)))

['img1', 'img10', 'img11', 'img12', 'img13', 'img14', 'img15', 'img17', 'img18', 'img19', 'img2', 'img21', 'img22', 'img23', 'img24', 'img25', 'img26', 'img27', 'img28', 'img3', 'img30', 'img31', 'img4', 'img5', 'img6', 'img7', 'img8', 'img9']

Total 28 folders


In [3]:
#WORKING CODE WITH ADDED FUNCTIONALITY# 

path=os.path.join('data/VOCdevkit2007/VOC2007/JPEGImages')  #this is the path where the images(from the dataset folder) are copied 

save_path='data/VOCdevkit2007/VOC2007/Annotations/' #this is the path where generated xml files are saved

index=0    #used for numbering of xml and images
objname=0 
col=[]
val=0    #used for numbering of dataframes(newdf)

maindf=pd.DataFrame()

############################### DATA PREPROCESSING AND EXTRACTING USEFUL VARIABLES #############################
for folder in imgset:
        #print('INDEX',index)
        img_path=os.path.join('dataset',folder)
        filename=img_path + '.csv'
        print(filename)
        df=pd.read_csv(filename, sep=' ')
        
        n=0
        
        v=list(df.columns)
        end=v.index(v[-1])
        df1=df.loc[:, v[1]:v[end]]
        #df1=df.iloc[:,1:]
        exp_dir=df.iloc[:,1:]
        numlist=df.imageFilename.apply(lambda x: x.split('/')[-1])
        
        #added code for negative annotation entries(bug) in MATLAB
        export=0
        for column in list(df1.columns):
            inx=0
            for value in list(df1[column]):
                if value<0 or value==0:
                    export=1
                    #df1.iloc[inx,list(df1.columns).index(column)]=1 The logic in line below can also be written like this
                    print('Non positive value', df1.loc[inx, column], ',changed to 1')
                    df1.loc[inx, column]=1
                    exp_dir.loc[inx, column]=1
                inx+=1
        
        #exporting corrected csv
        if export==1:
            exp_dir.insert(0, "imageFilename", numlist, True)
            exp_dir.to_csv(os.path.join('corr_csv',filename.split('/')[1]), index=None, header=True)
        
        data = df1.fillna(-1)
        data_int=data.astype(int)
        data_int.insert(0, "imageFilename", numlist, True)

        rowct=df.iloc[:,0].count()

        bld=0
        per=0
        veh=0
        wep=0
        colct=0
        classes=[]
        for x in v:
            x=x.split('_')[0]
            if x=='building':
                bld=bld+1
            elif x=='person':
                per=per+1
            elif x=='vehicle':
                veh=veh+1
            elif x=='weapon':
                wep=wep+1

        labels={'building':0,'person':0,'vehicle':0,'weapon':0}        

        labels['building']=bld
        labels['person']=per
        labels['vehicle']=veh
        labels['weapon']=wep

        for p,q in labels.items():
            if (q%4) == 0:
                colct=colct+q          
        colct=colct+1          # added 1 for imageFilename(extra column which is not included in labels dict)
        #print(colct)
        #print(bld,per,veh,wep)
        #print(labels)
        c=list(data_int.columns)
        for k in c:
            k=k.split('_')[0]
            classes.append(k)
        #len(classes)
        
        #added code for datasets where one or more classes are not present, which creates an empty column in csv file.
        for item in list(data_int.columns[1:]):
            try:
                item.split('_')[1]
            except:
                data_int.rename(columns={item:item+'_1'}, inplace=True)
        
        a=list(data_int.columns)

        fnames=list(numlist)
        indb=a.index('building_1')    
        indp=a.index('person_1')
        indv=a.index('vehicle_1')
        indw=a.index('weapon_1')

        blst=list(data_int.iloc[:, indb])
        plst=list(data_int.iloc[:, indp])
        vlst=list(data_int.iloc[:, indv])
        wlst=list(data_int.iloc[:, indw])
        nlst=list(data_int.iloc[:,0])
        
        newdf=pd.DataFrame({'imageFilename':nlst,'building_1':blst,'person_1':plst,'vehicle_1':vlst,'weapon_1':wlst})
        newdf = newdf[['imageFilename', 'building_1', 'person_1', 'vehicle_1', 'weapon_1']]  #rearranges the columns
        
        ############################################## XML CONVERT #####################################################
        while(n<rowct):
            col=data_int.iloc[n,:]        
            #noblist=list(data_int.iloc[n,:])
            #print(col)
            #print("Value of Column",n)
            #image_info = img_path + "/" + data_int.iloc[n,0]
            image_info = os.path.join(img_path, data_int.iloc[n,0])
            #print(image_info)
            #img_style = img_path.split('/')[-1]
            img_style = 'VOC2007'
            #img_name = data_int.iloc[n,0]
            img_name  = str(index) + '.jpg'
            
            #added a check so that only .jpeg images are permitted
            typ=imghdr.what(image_info)
            if typ!='jpeg':
                rem=nlst[n]        #rem contains the image that has to be removed because of invalid type(not a .jpeg)
                print('remove',nlst[n])
                newdf.drop(newdf[newdf['imageFilename'] == rem].index, axis=0, inplace=True)  #row containing invalid image type is dropped(removed)
                n=n+1
                continue

            root = etree.Element("annotation")
            
            img_size = Image.open(image_info).size
            img_width = img_size[0]
            img_height = img_size[1]
            img_depth = Image.open(image_info).layers
            
            folder = etree.Element("folder")
            folder.text = "%s" % (img_style)

            filename = etree.Element("filename")
            filename.text = "%s" % (img_name)

            #path = etree.Element("path")
            #path.text = "%s" % (img_path)

            source = etree.Element("source")
            ##################source - element##################
            source_database = etree.SubElement(source, "database")
            source_database.text = "The VOC2007 Database"
            source_annotation = etree.SubElement(source, "annotation")
            source_annotation.text = "PASCAL VOC2007"
            ####################################################

            size = etree.Element("size")
            ####################size - element##################
            image_width = etree.SubElement(size, "width")
            image_width.text = "%d" % (img_width)

            image_height = etree.SubElement(size, "height")
            image_height.text = "%d" % (img_height)

            image_depth = etree.SubElement(size, "depth")
            image_depth.text = "%d" % (img_depth)
            ####################################################

            segmented = etree.Element("segmented")
            segmented.text = "0"
            root.append(folder)
            root.append(filename)
            #root.append(path)
            root.append(source)
            root.append(size)
            root.append(segmented)
            
            #newdf.at[n, 'imageFilename'] = str(index) + '.jpg'
            
            i=1
            cl=1
            nob=0
            c=0

            bent=0
            cl=a.index('building_1')
            while(col[cl]!=-1 and classes[cl]=='building'):
                bent=bent+1
                cl=cl+1
            #print('bent',bent)

            pent=0
            cl=a.index('person_1')
            while(col[cl]!=-1 and classes[cl]=='person'):
                pent=pent+1
                cl=cl+1
            #print('pent',pent)

            vent=0
            cl=a.index('vehicle_1')
            while(col[cl]!=-1 and classes[cl]=='vehicle'):
                vent=vent+1
                cl=cl+1
            #print('vent',vent)


            went=0
            cl=a.index('weapon_1')
            while(cl<len(a) and col[cl]!=-1 and classes[cl]=='weapon'):
                went=went+1
                cl=cl+1
            #print('went',went)

            tnob=0

            while(i<len(col)):

                objname=classes[i]

                if objname=='building':
                    tnob=bent
                elif objname=='person':
                    tnob=pent
                elif objname=='vehicle':
                    tnob=vent
                elif objname=='weapon':
                    tnob=went
                if tnob%2!=0:
                    print('ERROR')

                #print('tnob',tnob)
                nob=int(tnob/4)
                #print(objname)
                #print('nob',nob)
                if a[i]=='building_1' or a[i]=='person_1' or a[i]=='vehicle_1' or a[i]=='weapon_1':
                    c=nob

                if col[i]==-1:
                    i=i+4
                    #if i<len(col):
                    #print('--------------------')
                    #c=nob

                else:

                    if (tnob==4):
                        #print(i)
                        #print('################################################')

                        objname=classes[i]
                        b_xmin=int(col[i])
                        #print(b_xmin)
                        b_ymin=int(col[i+1])
                        #print(b_ymin)
                        width=int(col[i+2])
                        #print(width)
                        height=int(col[i+3])
                        #print(height)
                        i=i+4
                        b_xmax=b_xmin+width
                        #print(b_xmax)
                        b_ymax=b_ymin+height
                        #print(b_ymax)

                    elif(tnob>4):

                        r=0
                        b_xmin=int(col[i])
                        #print('R',r)
                        #print(b_xmin)
                        r=int(i)
                        #print('R',r)
                        r=int(r+nob)
                        b_ymin=int(col[r])
                        #print('R',r)
                        #print(b_ymin)
                        r=int(r+nob)
                        width=int(col[r])
                        #print('R',r)
                        #print(width)
                        r=int(r+nob)
                        height=int(col[r])
                        #print('R',r)
                        #print(height)
                        i=i+1
                        #print('I with counter',i)
                        c=c-1
                        #print('C',c)
                        b_xmax=b_xmin+width
                        b_ymax=b_ymin+height
                        if c==0:
                            i=i-nob+tnob      #reset i
                            tnob=0
                            #print('changed I',i)

                    else:
                        objname=classes[i]

                        if objname=='building':
                            tnob=bent
                        elif objname=='person':
                            tnob=pent
                        elif objname=='vehicle':
                            tnob=vent
                        elif objname=='weapon':
                            tnob=went

                        if tnob%2!=0:
                            print('ERROR')
                        #print('tnob',tnob)
                        continue

                    object = etree.Element("object")
                    ####################object - element##################
                    name = etree.SubElement(object, "name")
                    name.text = "%s" % (objname)

                    pose = etree.SubElement(object, "pose")
                    pose.text = "Unspecified"

                    truncated = etree.SubElement(object, "truncated")
                    truncated.text = "0"

                    difficult = etree.SubElement(object, "difficult")
                    difficult.text = "0"

                    bndbox = etree.SubElement(object, "bndbox")
                    #####sub_sub########
                    xmin = etree.SubElement(bndbox, "xmin")
                    xmin.text = "%d" % (b_xmin)
                    ymin = etree.SubElement(bndbox, "ymin")
                    ymin.text = "%d" % (b_ymin)
                    xmax = etree.SubElement(bndbox, "xmax")
                    xmax.text = "%d" % (b_xmax)
                    ymax = etree.SubElement(bndbox, "ymax")
                    ymax.text = "%d" % (b_ymax)
                    #####sub_sub########
                    #print('I',i)
                    #print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')

                    root.append(object) #appends the object element to the xml file

            image = cv2.imread(os.path.join(img_path,fnames[n]))
            cv2.imwrite(os.path.join(path, str(index)+'.jpg'), image)
            
            if n==rowct-1:
                file_output = etree.tostring(root, encoding='UTF-8')
                ff = open('%s%s.xml' % (save_path, index), 'w', encoding="UTF-8")
                ff.write(file_output.decode('UTF-8'))

            file_output = etree.tostring(root, encoding='UTF-8')
            # print(file_output.decode('utf-8'))
            ff = open('%s%s.xml' % (save_path, index), 'w', encoding="UTF-8")
            ff.write(file_output.decode('UTF-8'))        #writes the XML file
            #print('File created')
            n=n+1
            index+=1
        ims=len(list(newdf.iloc[:,0]))
        print('Images in current folder',ims)
        
        #for renaming the images in csv files to match with xml and saved images
        for f in range(0,len(newdf.iloc[:,0])):
            newdf.iloc[f,0]=str(val)+'.jpg'
            val+=1

        maindf=pd.concat([maindf, newdf], ignore_index=True)
        t_ims=len(list(maindf.iloc[:,0]))
        print('Total images',t_ims)

dataset/img1.csv
Images in current folder 702
Total images 702
dataset/img10.csv
Images in current folder 496
Total images 1198
dataset/img11.csv
remove 2150.jpg
remove 3297.jpg
remove 3441.jpg
remove 3557.jpg
remove 3828.jpg
remove 401.jpg
remove 497.jpg
remove 5163.jpg
remove 5286.jpg
remove 5695.jpg
remove 6450.jpg
Images in current folder 6008
Total images 7206
dataset/img12.csv
remove 2107.jpg
remove 3780.jpg
remove 3994.jpg
remove 4699.jpg
remove 5066.jpg
remove 5330.jpg
remove 5702.jpg
remove 5738.jpg
remove 6140.jpg
remove 6930.jpg
remove 700.jpg
remove 7385.jpg
remove 7521.jpg
remove 7522.jpg
remove 7570.jpg
remove 8146.jpg
Images in current folder 7582
Total images 14788
dataset/img13.csv
Images in current folder 1316
Total images 16104
dataset/img14.csv
Images in current folder 898
Total images 17002
dataset/img15.csv
Non positive value -1.0 ,changed to 1
Non positive value 0.0 ,changed to 1
Non positive value 0.0 ,changed to 1
Non positive value 0.0 ,changed to 1
Non positi

remove 000000132710.jpg
remove 000000132719.jpg
remove 000000132724.jpg
remove 000000133110.jpg
remove 000000133532.jpg
remove 000000134033.jpg
remove 000000134098.jpg
remove 000000134285.jpg
remove 000000134648.jpg
remove 000000134703.jpg
remove 000000134760.jpg
remove 000000134981.jpg
remove 000000135155.jpg
remove 000000135229.jpg
remove 000000135258.jpg
remove 000000135263.jpg
remove 000000135472.jpg
remove 000000135585.jpg
remove 000000135754.jpg
remove 000000135984.jpg
remove 000000136267.jpg
remove 000000136683.jpg
remove 000000136870.jpg
remove 000000136971.jpg
remove 000000137120.jpg
remove 000000137332.jpg
remove 000000137501.jpg
remove 000000137654.jpg
remove 000000137992.jpg
remove 000000138675.jpg
remove 000000138742.jpg
remove 000000138789.jpg
remove 000000138963.jpg
remove 000000138975.jpg
remove 000000138982.jpg
remove 000000139000.jpg
remove 000000139139.jpg
remove 000000139151.jpg
remove 000000139261.jpg
remove 000000139539.jpg
remove 000000139614.jpg
remove 000000139

In [4]:
#EXPORT THE FILE#
maindf
exp_maindf=maindf.to_csv(r'exp.csv', index=None, header=True)